In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from mls_scf_tools.mls_pint import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]


In [ ]:
import intervaltree
import itertools
import pint
from matplotlib import colormaps
import warnings

In [ ]:
%autoreload 2
from njl_corf.pyfcctab import ureg
import njl_corf.pyfcctab as pyfcctab
import njl_corf.pyoscar as pyoscar

In [ ]:
allocation_database = pyfcctab.read()

In [ ]:
oscar_database = pyoscar.read()
oscar_database = pyoscar.merge_sensors(
    oscar_database,
    {
        "AMSR": "AMSR*",
        "ATMS": "ATMS*",
        "AWS": "AWS*",
        "MWTS": "MWTS*",
        "SSMIS": "SSMIS*",
        "AMSU": "AMSU*",
        #        "MWRI": "MWRI*",
        "MWS": "MWS*",
        "MTVZA": "MTVZA*",
        "MWI": "MWI*",
        "SRAL": "SRAL*",
        "OSCAT": "OSCAT*",
        "Altimetry": ["ALT*", "RA*", "Altimeter*"],
        "Precipitation": ["PR*", "DPR*", "Rainradar"],
        "Poseidon": "Poseidon*",
        "SWIM": "SWIM*",
        "MWR": "MWR*",
        "WSF": "WSF*",
        "MLS": "MLS*",
        "ICI": "ICI*",
    },
)

In [ ]:
def views_plot(
    frequency_range: slice,
    allocation_database: pyfcctab.FCCTables,
    oscar_database: intervaltree.IntervalTree,
    ax: matplotlib.pyplot.Axes,
    frequency_margin: float = None,
):
    """Does one views report plot"""
    # Increate the range if/as needed
    if frequency_margin is None:
        frequency_margin = 0.20
    formal_bandwidth = frequency_range.stop - frequency_range.start
    examined_frequency_range = slice(
        frequency_range.start - formal_bandwidth * frequency_margin,
        frequency_range.stop + formal_bandwidth * frequency_margin,
    )
    # ------ Allocations
    bands = allocation_database.itu[examined_frequency_range]
    # Work out how many services are in use
    primary_allocations = itertools.chain.from_iterable(
        [band.primary_allocations for band in bands]
    )
    secondary_allocations = itertools.chain.from_iterable(
        [band.secondary_allocations for band in bands]
    )
    allocations = list(primary_allocations) + list(secondary_allocations)
    services = sorted(list(set(allocation.service.name for allocation in allocations)))
    n_services = len(services)
    # Loop over the services we found and include them in the plot
    y_labels = {}
    colors = colormaps["tab10"].colors
    band_bounds = []
    for i_service, service in enumerate(services):
        y_labels[i_service] = service
        for band in bands:
            band_bounds += [band.bounds[0], band.bounds[1]]
            if band.has_allocation(service + "*"):
                x_bounds = (
                    pint.Quantity.from_sequence(band.bounds)
                    .to(frequency_range.start.units)
                    .magnitude
                )
                thickness = 0.2
                y_bounds = np.array([i_service - thickness, i_service + thickness])
                xy = [x_bounds[[0, 1, 1, 0]], y_bounds[[0, 0, 1, 1]]]
                this_patch = matplotlib.patches.Polygon(
                    np.stack(xy, axis=1),
                    color=colors[i_service % len(colors)],
                    linewidth=0,
                )
                ax.add_patch(this_patch)
                if band.has_allocation(service + "*", secondary=True):
                    this_hatch_patch = matplotlib.patches.Polygon(
                        np.stack(xy, axis=1),
                        color="white",
                        fill=None,
                        hatch="/////",
                        linewidth=0,
                    )
                    ax.add_patch(this_hatch_patch)
    # ------- Oscar
    # Now do all the things found in OSCAR
    eess_usage = oscar_database[examined_frequency_range]
    # Draw bars for each band usage
    for i, eess_user in enumerate(eess_usage):
        y_value = i + n_services
        y_labels[y_value] = eess_user.data.service
        x_bounds = (
            pint.Quantity.from_sequence(
                [eess_user.data.bounds.start, eess_user.data.bounds.stop]
            )
            .to(frequency_range.start.units)
            .magnitude
        )
        if x_bounds[1] > x_bounds[0] + 1e-6:
            ax.plot(
                x_bounds,
                [y_value] * 2,
                color="black",
                linewidth=2.0,
            )
        else:
            ax.plot(x_bounds[0], y_value, color="black", marker="*")
    # ------- Remainder
    # Now set up the rest of the plot information
    n_bars = n_services + len(eess_usage)
    ax.set_xlim(examined_frequency_range.start, examined_frequency_range.stop)
    ax.set_ylim(-0.5, n_bars - 0.5)
    ax.set_yticks(np.array(list(y_labels.keys())), y_labels.values())
    band_bounds = list(set(band_bounds))
    for b in band_bounds:
        ax.axvline(b, color="darkgrey", zorder=-10, linewidth=0.5)
    # ------- 5.340
    # Hatch out all the areas that are 5.340
    for band in bands:
        if band.has_footnote("5.340"):
            x_bounds = (
                pint.Quantity.from_sequence(band.bounds)
                .to(frequency_range.start.units)
                .magnitude
            )
            y_bounds = np.array(ax.get_ylim())
            xy = [x_bounds[[0, 1, 1, 0]], y_bounds[[0, 0, 1, 1]]]
            this_patch = matplotlib.patches.Polygon(
                np.stack(xy, axis=1),
                linewidth=0,
                facecolor="lightgrey",
                zorder=-10,
            )
            ax.add_patch(this_patch)

In [ ]:
# help(matplotlib.patches.Polygon)

In [ ]:
ranges = {
    "1.1": slice(47.2 * ureg.GHz, 51.4 * ureg.GHz),
    "1.2": slice(13.5 * ureg.GHz, 14.6 * ureg.GHz),
    "1.3": slice(51.4 * ureg.GHz, 52.4 * ureg.GHz),
    "1.4": slice(17.3 * ureg.GHz, 17.7 * ureg.GHz),
    "1.5": None,
    "1.6ab": slice(37.5 * ureg.GHz, 43.5 * ureg.GHz),
    "1.6cd": slice(47.2 * ureg.GHz, 51.4 * ureg.GHz),
    "1.7a": slice(4_400 * ureg.MHz, 4_800 * ureg.MHz),
    "1.7b": slice(7_125 * ureg.MHz, 8_400 * ureg.MHz),
    "1.7c": slice(14.8 * ureg.GHz, 15.35 * ureg.GHz),
    "1.8": slice(231.5 * ureg.GHz, 275 * ureg.GHz),
    "1.9": None,
    "1.10a": slice(71 * ureg.GHz, 76 * ureg.GHz),
    "1.10b": slice(81 * ureg.GHz, 86 * ureg.GHz),
    "1.11abcde": slice(1_518 * ureg.MHz, 1_675 * ureg.MHz),
    "1.11f": slice(2_483.5 * ureg.MHz, 2_500 * ureg.MHz),
    "1.12a": slice(1_427 * ureg.MHz, 1_432 * ureg.MHz),
    "1.12b": slice(1_645.5 * ureg.MHz, 1_646.5 * ureg.MHz),
    "1.12c": slice(1_880 * ureg.MHz, 1_920 * ureg.MHz),
    "1.12d": slice(2_010 * ureg.MHz, 2_025 * ureg.MHz),
    "1.13": None,
    "1.14": None,
    "1.15": None,
    "1.16": None,
    "1.17": None,
    "1.18": None,
    "1.19a": slice(4_200 * ureg.MHz, 4_400 * ureg.MHz),
    "1.19b": slice(8_400 * ureg.MHz, 8_500 * ureg.MHz),
}

In [ ]:
# allocation_database.footenote_definitions[0]["5.340"]

In [ ]:
ureg.setup_matplotlib()
fig, ax = plt.subplots(figsize=[4, 8])  # , layout="constrained")
test = views_plot(
    frequency_range=ranges["1.1"],
    allocation_database=allocation_database,
    oscar_database=oscar_database,
    ax=ax,
)
# plt.savefig("test.pdf")

In [ ]:
bands = allocation_database.itu[183 * ureg.GHz]
print([band.allocations[0].modifiers for band in bands])

In [ ]:
??allocation_database.itu.data.split_overlaps

In [ ]:
x = slice(1, 2)
help(x)